In [ ]:
import numpy as np
import pandas as pd
import csv
import random
import copy

In [ ]:
random.seed(2)

In [ ]:
# Import the data_50.txt file
from google.colab import files
uploaded = files.upload()

file = 'input_20_4_19.txt'
conflicts = []
with open(file) as f:
    N = int(f.readline())
    d = [int(s) for s in f.readline().split()]
    M = int(f.readline())
    c = [int(s) for s in f.readline().split()]
    K = int(f.readline())
    for i in range(K):
        conflicts.append([int(s) for s in f.readline().split()])


In [ ]:
conflict = []
for i in range(N+1):
    conflict.append(set())

In [ ]:
for conf in conflicts:
    conflict[conf[0]].add(conf[1])
    conflict[conf[1]].add(conf[0])

In [ ]:
for i in range(len(conflict)):
    print(i, ": ", conflict[i])

In [ ]:
# N = 20
# d = []
# M = 10
# K = 15
# conflicts = []
# for _ in range(N):
#     d.append(random.randint(70, 100))
# c = [random.randint(50, 70) for _ in range(M)]
# for _ in range(K):
#     conflict = np.random.randint(N, size=2)
#     # conflicts.append([d[conflict[0]], d[conflict[1]]])
#     conflicts.append([conflict[0], conflict[1]])


In [ ]:
subjectsList = []
binarySubject=[]
for i in range(N):
    subjectsList.append(i+1)
    binarySubject.append('{0:010b}'.format(i))

subjects = pd.DataFrame(list(zip(subjectsList, d, binarySubject)), 
                       columns = ['Subject', 'Capacity', 'Binary'])

subjects


,Subject,Capacity,Binary
0,1,5592,0000000000
1,2,776,0000000001
2,3,1584,0000000010
3,4,443,0000000011
4,5,305,0000000100
...,...,...,...
95,96,3647,0001011111
96,97,1206,0001100000
97,98,2178,0001100001
98,99,7434,0001100010


In [ ]:
RoomsLiss = []
binaryRoom=[]
for i in range(M):
    RoomsLiss.append(i+1)
    binaryRoom.append('{0:010b}'.format(i))

rooms = pd.DataFrame(list(zip(RoomsLiss, c, binaryRoom)), 
                       columns = ['Class', 'Capacity', 'Binary'])

rooms


In [ ]:
subjectConfList1 = [conflicts[i][0] for i in range(K)]
subjectConfList2 = [conflicts[i][1] for i in range(K)]
binaryConf = []
for i in range(K):
    binaryConf.append('{0:010b}'.format(i))

subjectConf = pd.DataFrame(list(zip(subjectConfList1, subjectConfList2, binaryConf)), 
                       columns = ['Subject1', 'Subject2', 'Binary'])
subjectConf

In [ ]:
# Exam class
class Exam:
    def __init__(self, timeSlot, subject, room):
        self.timeSlot = timeSlot
        self.subject = subject
        self.room = room
    def __repr__(self):
        return ('(TimeSlot: {0}, Subject: {1}, Room: {2})'.format(self.timeSlot, self.subject, self.room))

In [ ]:
# Class Individual
class Individual:
    def __init__(self, chromosome, fitness1, fitness2, h1=0, h2=0, h3=0, o = 0):
        self.chromosome = chromosome
        self.fitness1 = fitness1
        self.fitness2 = fitness2
        self.h1 = h1
        self.h2 = h2
        self.h3 = h3
        self.o = o
    def __repr__(self):
        return ('(Chromosome: {0}, Fitness1: {1}, Fitness2: {2})'.format(self.chromosome, self.fitness1, self.fitness2))

In [ ]:
TIMESLOTS = N

In [ ]:
#generate a string of binary values to represent one exam
#time slot, subject, room, multiroom

def getRandomExam(subject, rooms, subjectConf, sameRooms):
    timeSlots = set()
    # sameRooms = dict()
    timeSlot = '{0:010b}'.format(random.randint(1, TIMESLOTS))

    room = random.choice(rooms['Binary'].tolist())
    while room in list(sameRooms[int(timeSlot,2)]):
        timeSlot = '{0:010b}'.format(random.randint(1, TIMESLOTS))
        room = random.choice(rooms['Binary'].tolist())
        
    timeSlots.add(timeSlot)
    
    sameRooms[int(timeSlot,2)].add(room)
    exam = timeSlot + subject + room 
    
    students = subjects.at[list(np.where(subjects['Binary']==subject)[0])[0], 'Capacity']
    capacityRoom = rooms.at[list(np.where(rooms['Binary']==room)[0])[0], 'Capacity']
    restCapacitySubject = students - capacityRoom
    while restCapacitySubject > 0:
        if (len(timeSlots) < 4):
            timeSlotNext = '{0:010b}'.format(random.randint(1, TIMESLOTS))
            
            roomNext = random.choice(rooms['Binary'].tolist())    
            while roomNext in list(sameRooms[int(timeSlotNext,2)]):
                timeSlotNext = '{0:010b}'.format(random.randint(1, TIMESLOTS))
                roomNext = random.choice(rooms['Binary'].tolist())
    
            timeSlots.add(timeSlotNext)
        else:
            timeSlotNext = random.choice(list(timeSlots))

            roomNext = random.choice(rooms['Binary'].tolist())    
            while roomNext in list(sameRooms[int(timeSlotNext,2)]):
                timeSlotNext = random.choice(list(timeSlots))
                roomNext = random.choice(rooms['Binary'].tolist())
    
        # timeSlots.add(timeSlotNext)
        sameRooms[int(timeSlotNext,2)].add(roomNext)

        restCapacitySubject -= rooms.at[list(np.where(rooms['Binary']==roomNext)[0])[0], 'Capacity']
        exam += timeSlotNext + subject + roomNext

    return exam


In [ ]:
#given an exam, randomly generate its teacher, classroom, starting hour and day
def convertToChromosome(exam):
    timeSlot = '{0:010b}'.format(random.randint(1, TIMESLOTS))
    # room = random.choice(rooms['Binary'].tolist())
    subject = exam[10:20]
    exam = timeSlot + subject
    return exam

In [ ]:
# To convert chromosome back to Exam form
def convertToExamObject(exam):
    if (len(list(np.where(subjects['Binary']==exam[10:20])[0]))<=0 ):
        return None
    # subject = subjects.at[list(np.where(subjects['Binary']==exam[10:20])[0])[0], 'Subject']
    subject = int(exam[10:20], 2) + 1
    # room = int(exam[20:30], 2) + 1
    timeSlot = int(exam[0:10], 2)
    
    return Exam(timeSlot, subject)

In [ ]:
# Get individual exam strings from schedule
def getExamStrings(schedule):
    exams = []
    i = 0
    while (i < len(schedule)):
        exams.append(schedule[i:i+20])
        i += 20
    return exams

In [ ]:
def generatePopulation(populationSize, subjects, rooms, subjectConf):
    population = []
    for x in range(populationSize):
        schedule=[]
        sameRooms = []
        for i in range(TIMESLOTS + 1):
            sameRooms.append(set())

        timeSlotConflict = dict()
        for i in range(N):
            timeSlotConflict[i+1] = []

        CourseListCount = [0 for _ in range(len(subjects))]

        for i in range(len(subjects)):
            subject = subjects['Subject'][i]

            while True:
                print(i+1)
                temp = getRandomExam(subjects['Binary'][i], rooms, subjectConf, sameRooms)
                lenTemp = len(temp)
                timeSlot = []
                check = True
                while (lenTemp>=30):
                    t = temp[(lenTemp-30): (lenTemp-20)]
                    timeSlot.append(t)
                    lenTemp -=30

                b = True
                for x in timeSlot:
                    # t = int(x, 2)
                    for k in conflict[subject]:
                        if x in timeSlotConflict[k]:
                            b = False
                            break
                    if b == False:
                        break
                if b:
                    # for x in timeSlot:
                    #     timeSlotConflict[subject].append(x)
                    # schedule.append(temp)
                    # break
                    l = len(temp)
                    while l >=30:
                        schedule.append(temp[l-30:l-10])
                        l -= 30
                    break
        
        schedulestring = ""
        for e in schedule:
            schedulestring += e
        population.append(Individual(schedulestring, -1, -1))
    return population

new_population = generatePopulation(10, subjects, rooms, subjectConf)
print(new_population)

for s in new_population:
    exams=getExamStrings(s.chromosome)
    sortedExams = sorted(exams, key=lambda x: (int(x[0:10], 2)))
    chromosome = ""
    for e in sortedExams:
        chromosome += e
    s.chromosome = chromosome

for s in new_population:
    exams=getExamStrings(s.chromosome)
    print("*****************Schedule Option*******************")
    for e in exams:
        print(convertToExamObject(e))

In [ ]:
#roulette wheel selection
def rouletteWheelSelection(population):
  totalFitness=sum([p.fitness2 for p in population])
  individualProbabilities=[(p.fitness2)/totalFitness for p in population]

  return population[np.random.choice(len(population), p=individualProbabilities)]


In [ ]:
def getFitness(schedule):
    noOfConflicts = 0
    missingExams = 0
    subjectConfs = 0
    sameRooms = 0

    listOfExamsStrings = getExamStrings(schedule.chromosome) # exams
    listOfExams = []
    for e in listOfExamsStrings:
        listOfExams.append(convertToExamObject(e))
    uniqueExams = {e.subject for e in listOfExams}
    uniqueExams = list(uniqueExams)

    # check that all subjects scheduled
    # capacity = [0]*(N+1)
    # for e in listOfExams:
    #     capacity[e.subject] += rooms.at[list(np.where(rooms['Class']==e.room)[0])[0], 'Capacity']
    # for e in uniqueExams:
    #     if capacity[e] < subjects.at[list(np.where(subjects['Subject']==e)[0])[0], 'Capacity']:
    #         missingExams += 1

    subjs = subjects['Subject'].tolist()

    missingExams += abs(len(subjs) - len(uniqueExams))

    # Get the subjects schedule at the same timeSlot
    sameTimes = [[]]
    examsCopy = copy.deepcopy(listOfExams)  
    for e, eobj in enumerate(examsCopy):
        timeSlot = eobj.timeSlot
        templist = []
        for e2, e2obj in enumerate(examsCopy):
            if (e2obj.timeSlot == timeSlot and e2obj.timeSlot != -1):
                e2copyObj = copy.deepcopy(e2obj)
                templist.append(e2copyObj)
                examsCopy[e2].timeSlot = -1
        sameTimes.append(templist)

    # Check that subjects conficts, same room
    for sameTime in sameTimes:
        if len(sameTime) > 1:
            # Check that subjects conficts
            for i in range(len(sameTime)):
                exam1 = sameTime[i]
                subject1 = exam1.subject
                subj1Conf = conflict[subject1]
            
                for j in range(i+1, len(sameTime)):
                    exam2 = sameTime[j]
                    subject2 = exam2.subject
                    if ((subject2 != subject1) and (subject2 in list(subj1Conf))):
                        subjectConfs +=1
                    # if (exam1.room == exam2.room):
                    #     sameRooms += 1

                # # Check that same room
                # exam1 = sameTime[i]
                # for k in range(len(sameTime)):
                #     exam2 = sameTime[k]
                #     if exam1.subject == exam2.subject and  exam1.room == exam2.room and exam1 != exam2:
                #         sameRooms += 1
    
    uniqueTimeSlot = {e.timeSlot for e in listOfExams}
    timeSlots = len(uniqueTimeSlot)

    noOfConflicts += missingExams
    schedule.h1 = missingExams

    noOfConflicts += subjectConfs
    schedule.h2 = subjectConfs

    # noOfConflicts += sameRooms
    # schedule.h3 = sameRooms

    schedule.o = timeSlots
  
    schedule.fitness1 = 1/(float(noOfConflicts) + 1)
    schedule.fitness2 = 1/(float(timeSlots))
    return noOfConflicts
    

In [ ]:
for p in new_population:
    getFitness(p)
    print(p.h1)
    print(p.h2)
    print(p.h3)

In [ ]:
def crossover(best, second_best):
    crossed_exams_two = []
    crossed_exams_one = []

    # Size of one exam is = 21 bits
    exams_one = getExamStrings(best.chromosome)
    exams_two = getExamStrings(second_best.chromosome)

    exams = exams_one if len(exams_one) < len(exams_two) else exams_two

    for i, val in enumerate(exams):
        if (i%2 == 0):
            exam_cross_one= val[0:10] + exams_two[i][10:20]
        else:
            exam_cross_one = exams_two[i][0:10] + val[10:20]
        
        crossed_exams_one.append(exam_cross_one)
    
    crossover_schedule = Individual("", -1, -1)
    crossover_schedule.chromosome = ""

    for x in crossed_exams_one:
        crossover_schedule.chromosome += x
    return crossover_schedule


In [ ]:
def mutation(best, mutation_probability):
    exams_one = getExamStrings(best.chromosome)
    for exam in exams_one:
        
    listOfExams = []
    for e in exams_one:
        listOfExams.append(convertToExamObject(e))
    sameTimes = [[]]
    for e, eobj in enumerate(listOfExams):
        timeSlot = eobj.timeSlot
        templist = []
        for e2, e2obj in enumerate(listOfExams):
            if (e2obj.timeSlot == timeSlot and e2obj.timeSlot != -1):
                e2copyObj = copy.deepcopy(e2obj)
                templist.append(e2copyObj)
                listOfExams[e2].timeSlot = -1
        sameTimes.append(templist)

    for i in range(len(exams_one)):
        if (mutation_probability > random.random()):
            exams_one[i] = convertToChromosome(exams_one[i])
        
    best.chromosome = ""
    for e in exams_one:
        best.chromosome += e
    
    return best


In [ ]:
def getSortedFitness(population):
    population = sorted(population, key = lambda x: (x.fitness1 + x.fitness2))
    return population

In [ ]:
def crossover_population(population, size):
    population = getSortedFitness(population)
    new_crossed_population = []
    new_crossed_population.append(population[-1])

    i = 1
    while (i < size):
        fittest = copy.deepcopy(rouletteWheelSelection(population))
        second_fittest = copy.deepcopy(rouletteWheelSelection(population))
        new_schedule = crossover(fittest, second_fittest)
        getFitness(new_schedule)

        # add the new crossed schedule only if it has a better fitness value
        if new_schedule.fitness2 > fittest.fitness2 and new_schedule.fitness2 > second_fittest.fitness2:
            # check if extra exams have been created for any course the perform crossover again
            exams = getExamStrings(new_schedule.chromosome)

            makeCont = 0
            for e in exams:
                examObj = convertToExamObject(e)
                for e2 in exams:
                    examObj2 = convertToExamObject(e2)

                    if e != e2 and examObj.subject == examObj2.subject:
                        makeCont = 1
                        break
                if makeCont == 1:
                    break
            if makeCont:
                continue
            
            new_crossed_population.append(new_schedule)
        elif fittest.fitness2 > new_schedule.fitness2 and fittest.fitness2 > second_fittest.fitness2:
            new_crossed_population.append(fittest)
        else:
            new_crossed_population.append(second_fittest)
        i+=1
    return new_crossed_population


In [ ]:
def mutate_population(population, size, prob):
    for i in range(1, size):
        current_mutated = copy.deepcopy(population[i])
        current_mutated = mutation(current_mutated, prob)
        getFitness(current_mutated)
        population[i] = copy.deepcopy(current_mutated)
    return population

In [ ]:
def propogate(population, size, prob):
    return mutate_population(crossover_population(population, size), size, prob)

In [ ]:
def GA(size):
    mutation_prob = 0.2
    hardconstraints = 0
    check = 0
    population = new_population
    for p in population:
        getFitness(p)
    # print("The maximum fitness1 at start is ", population[-1].fitness1)
    # print("The maximum fitness2 at start is ", population[-1].fitness2)

    for i in range(200):
        population = propogate(population, size, mutation_prob)
        population = getSortedFitness(population)
        # print("The maximum fitness1 uptil now is ", population[-1].fitness1)
        print("The maximum fitness2 uptil now is ", population[-1].fitness2)

        if (population[-1].fitness1 > 0.5 and hardconstraints == 0):
            # print("\U0001f600", "Schedule with all hard constraints satisfied:")
            # print("\Chromosome: ", population[-1].chromosome)
            exams = getExamStrings(population[-1].chromosome)
            examList = []

            for e in exams:
                examList.append(convertToExamObject(e))

            sortedExams = sorted(examList, key=lambda x: x.timeSlot)
            # for e in sortedExams:
            #     print(e)
            hardconstraints = 1

            # print("List of hard constraints satisified: ")
            # if (population[-1].h1 == 0):
            #     print("An exam will be scheduled for each course")
            
            # if (population[-1].h2 == 0):
            #     print("No subject are in conflict")

            # if (population[-1].h3 == 0):
            #     print("Cannot two exams at the sam room")

        # if (hardconstraints==1):
        #     check +=1

    return population[-1]

In [ ]:
schedule = GA(10)
print("\U0001f600", "The most optimum schedule: **")
print("Chromosome: ", schedule.chromosome) 
exams=getExamStrings(schedule.chromosome)
examList=[]
for e in exams:
  examList.append(convertToExamObject(e))

sortedExams = sorted(examList, key=lambda x: (x.timeSlot))
for e in sortedExams:
  print(e)

print("List of hard constraints satisifed: ")
if (schedule.h1 == 0):
    print("An exam will be scheduled for each course")     
if (schedule.h2 == 0):
    print("No subject are in conflict")
if (schedule.h3 == 0):
    print("Cannot two exams at the sam room")
print("TimeSlots: ", schedule.o)

In [ ]:
sortedExams = sorted(sortedExams, key=lambda x: (x.subject))
for e in sortedExams:
  print(e)